In [15]:
!pip install deap

In [16]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from deap import base, creator, tools, algorithms
import random
import numpy as np
import time
import matplotlib.pyplot as plt

In [17]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Step 1: Environment Setup
# Define multi-objective fitness to maximize accuracy, minimize complexity, and minimize training time
creator.create("FitnessMulti", base.Fitness, weights=(1.0, -1.0, -1.0))  # Maximize accuracy, minimize complexity & time
creator.create("Individual", list, fitness=creator.FitnessMulti, generation=int)

/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [18]:
# Step 2: Data Preparation
def load_data():
    print("Loading and preprocessing CIFAR-10 dataset...")
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    # Use a smaller subset of data for quicker training
    subset_size = 1000  # Use only 1,000 samples for speed
    x_train = x_train[:subset_size].astype('float32') / 255.0
    y_train = to_categorical(y_train[:subset_size], 10)

    # Use a small subset of the validation data as well
    x_val = x_train[:200]  # 200 samples for validation
    y_val = y_train[:200]

    return (x_train, y_train), (x_val, y_val)

In [19]:
# Step 3: Initial Population Generation
def generate_individual(gen=0):
    layers = []
    num_layers = random.randint(2, 3)  # Limit number of layers for faster training
    for _ in range(num_layers):
        layer_type = random.choice(['conv', 'dense'])
        if layer_type == 'conv':
            layers.append({
                'type': 'conv',
                'filters': random.choice([16, 32]),
                'activation': random.choice(['relu', 'tanh'])
            })
        elif layer_type == 'dense':
            layers.append({
                'type': 'dense',
                'units': random.choice([64, 128]),
                'activation': random.choice(['relu', 'tanh'])
            })
    individual = creator.Individual(layers)
    individual.generation = gen  # Set initial generation to 0
    return individual

toolbox = base.Toolbox()
toolbox.register("individual", generate_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [20]:
# Step 4: Implement Evaluation Function
def create_cnn(architecture):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(32, 32, 3)))  # Input layer for CIFAR-10

    added_flatten = False
    for layer in architecture:
        if layer['type'] == 'conv' and not added_flatten:
            model.add(tf.keras.layers.Conv2D(layer['filters'], (3, 3), activation=layer['activation'], padding='same'))
            model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        elif layer['type'] == 'dense':
            if not added_flatten:
                model.add(tf.keras.layers.Flatten())
                added_flatten = True
            model.add(tf.keras.layers.Dense(layer['units'], activation=layer['activation']))

    if not added_flatten:
        model.add(tf.keras.layers.Flatten())

    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [26]:
def evaluate_cnn(individual):
    print(f"Evaluating individual from Generation {individual.generation}: {individual}")
    model = create_cnn(individual)
    start_time = time.time()

    # Train on a smaller subset of data for faster evaluation
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val),
                        epochs=2, batch_size=16, verbose=0)  # Reduce epochs and batch size
    accuracy = model.evaluate(x_val, y_val, verbose=0)[1]
    model_complexity = model.count_params()
    training_time = time.time() - start_time

    return accuracy, model_complexity, training_time

toolbox.register("evaluate", evaluate_cnn)

In [27]:
# Step 5: Lexicase Selection with generation tracking
def lexicase_selection(population, k):
    selected = []
    for _ in range(k):
        candidates = population[:]
        random.shuffle(candidates)
        objectives = list(range(len(population[0].fitness.values)))
        random.shuffle(objectives)
        for obj in objectives:
            best_candidates = [c for c in candidates if c.fitness.values[obj] == max(c.fitness.values[obj] for c in candidates)]
            candidates = best_candidates
            if len(candidates) == 1:
                break
        selected_individual = candidates[0]
        selected.append(selected_individual)
        print(f"Selected Individual from Generation {selected_individual.generation}: {selected_individual}")
    return selected

toolbox.register("select_lexicase", lexicase_selection)

In [28]:
# Step 6: Crossover and Mutation with generation updating
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)

def apply_crossover_and_mutation(offspring, cxpb, mutpb, gen):
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < cxpb:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values
            child1.generation = gen  # Update generation
            child2.generation = gen  # Update generation

    for mutant in offspring:
        if random.random() < mutpb:
            toolbox.mutate(mutant)
            del mutant.fitness.values
            mutant.generation = gen  # Update generation

In [32]:
# Step 7: Iterative Evolution Process with lexicase selection
def evolutionary_algorithm(n_gen=30, pop_size=10, cxpb=0.5, mutpb=0.2):
    population = toolbox.population(n=pop_size)
    fitness_log = []

    # Set the selection size to be half of the population size
    selection_size = pop_size // 2

    for gen in range(n_gen):
        print(f"\n-- Generation {gen} --")

        # Evaluate fitness for each individual
        fitnesses = list(map(toolbox.evaluate, population))
        for ind, fit in zip(population, fitnesses):
            ind.fitness.values = fit

        # Print all individuals and their fitness values with generation info
        print(f"\nPopulation for Generation {gen}:")
        for ind in population:
            print(f"Individual: {ind}, Generation: {ind.generation}, Fitness (Acc, Comp, Time): {ind.fitness.values}")

        avg_fitness = np.mean([ind.fitness.values[0] for ind in population])
        fitness_log.append(avg_fitness)
        print(f"\nAverage Accuracy: {avg_fitness}")

        # Selection using lexicase with constant selection size
        selected_parents = toolbox.select_lexicase(population, selection_size)

        # Print the selected parents
        print(f"\nSelected Parents for Generation {gen} using Lexicase Selection:")
        for parent in selected_parents:
            print(f"Parent: {parent}, Generation: {parent.generation}, Fitness (Acc, Comp, Time): {parent.fitness.values}")

        # Create offspring and apply crossover and mutation
        offspring = list(map(toolbox.clone, selected_parents))
        apply_crossover_and_mutation(offspring, cxpb, mutpb, gen + 1)
        population[:] = offspring

    # Plot evolutionary progress
    plt.plot(range(n_gen), fitness_log)
    plt.xlabel('Generation')
    plt.ylabel('Average Accuracy')
    plt.title('Evolutionary Progress')
    plt.show()

    best_ind = tools.selBest(population, 1)[0]
    return best_ind


### Generation 0

#### Initial Evaluation
- In Generation 0, five individuals are randomly generated, each representing a different neural network architecture.
- Each individual is evaluated based on three fitness metrics: **accuracy**, **complexity** (total number of model parameters), and **training time**.
- After evaluating all individuals, the algorithm calculates the **average accuracy** of the population, which is around **0.35**.

#### Lexicase Selection
- Lexicase selection is applied to select parents for the next generation based on a **greedy approach**, focusing on different objectives at each iteration.
- In this case, all selected parents share the same architecture: a **convolutional layer with 32 filters** followed by a **dense layer with 128 units**, which has the highest accuracy among Generation 0 individuals.
- This results in some repetition among selected parents, as they have the best combination of accuracy, complexity, and training time.

---

### Generation 1

#### Population Composition
- The new generation contains a mix of **new offspring** (produced by crossover and mutation of Generation 0 parents) and **unchanged individuals** from Generation 0 (due to elitism).
- This explains why some individuals from **Generation 0** still appear in the **Generation 1** population.

#### Performance Improvement
- The average accuracy increases to around **0.56**, indicating that the evolutionary process is finding **better architectures**.
- Lexicase selection once again favors individuals with the best performance, selecting architectures that showed strong results in Generation 1.

---

### Generation 2

#### Population Composition
- Similar to Generation 1, Generation 2 includes a mix of individuals from **Generation 1** (due to elitism) and **new offspring**.
- The average accuracy decreases slightly from **0.56 to around 0.53**, suggesting that while some offspring performed well, others might have degraded performance.

#### Best Individual
- The best architecture remains consistent across generations: **Conv2D with 32 filters** followed by **Dense with 128 units**, demonstrating robust performance.

---

### Graphical Output (Line Plot)
- The line plot illustrates **evolutionary progress** over three generations, with **average accuracy** on the y-axis and **generation number** on the x-axis.
- It shows an initial **steep rise** in accuracy from Generation 0 to Generation 1, reflecting effective selection and improvement.
- From Generation 1 to Generation 2, there’s a **slight drop in accuracy**, which is common in evolutionary algorithms due to exploration of new architectures that may not always perform better.

---

### Key Observations

#### Population Composition Across Generations
- The inclusion of individuals from previous generations is due to **elitism**, where the best individuals are preserved to maintain good solutions and prevent performance drops.
- This is crucial for maintaining **performance stability** and avoiding drastic declines in accuracy.

#### Selection Pressure and Convergence
- Lexicase selection applies **dynamic selection pressure** based on different objectives, often resulting in convergence towards the best architecture.
- This leads to **one architecture dominating** the population, as seen with the **Conv2D-32, Dense-128** structure.
- The slight decline in accuracy in Generation 2 may be due to the introduction of new offspring that don’t perform as well, emphasizing the balance between **exploration and exploitation** in evolutionary algorithms.

#### Performance Evolution
- The overall trend shows an **increase in average accuracy** from Generation 0 to Generation 1, followed by a slight decline.
- **Average accuracy** represents the fitness of the entire population, so a slight decline does not necessarily indicate poor evolutionary performance—it may indicate **diversity** in the population.


In [33]:
# Main execution
if __name__ == "__main__":
    (x_train, y_train), (x_val, y_val) = load_data()
    best_architecture = evolutionary_algorithm()
    print("\nBest architecture found:", best_architecture)

Loading and preprocessing CIFAR-10 dataset...

-- Generation 0 --
Evaluating individual from Generation 0: [{'type': 'conv', 'filters': 16, 'activation': 'relu'}, {'type': 'dense', 'units': 128, 'activation': 'relu'}, {'type': 'dense', 'units': 64, 'activation': 'tanh'}]
Evaluating individual from Generation 0: [{'type': 'dense', 'units': 64, 'activation': 'relu'}, {'type': 'dense', 'units': 128, 'activation': 'relu'}, {'type': 'conv', 'filters': 32, 'activation': 'relu'}]
Evaluating individual from Generation 0: [{'type': 'conv', 'filters': 32, 'activation': 'tanh'}, {'type': 'conv', 'filters': 16, 'activation': 'relu'}, {'type': 'conv', 'filters': 16, 'activation': 'tanh'}]
Evaluating individual from Generation 0: [{'type': 'dense', 'units': 64, 'activation': 'relu'}, {'type': 'dense', 'units': 128, 'activation': 'relu'}]
Evaluating individual from Generation 0: [{'type': 'conv', 'filters': 16, 'activation': 'relu'}, {'type': 'dense', 'units': 128, 'activation': 'relu'}, {'type': 'co

KeyboardInterrupt: 